# Solicionador de Helmholtz con el método de elementos de frontera

In [2]:
import numpy as np
from numpy.linalg import norm
import matplotlib.pyplot as plt
from scipy.special import hankel1

### 📐 Relación entre número de onda y frecuencia

La relación entre el número de onda $k$ y la frecuencia $f$ está dada por:

$$
f = \frac{k c}{2 \pi}
$$

donde:

- $f$: frecuencia en **Hertz (Hz)**,  
- $k$: número de onda en **radianes por metro (rad/m)**,  
- $c$: velocidad del sonido en **metros por segundo (m/s)**.

### Conversión de frecuencia a número de onda

Dada una frecuencia $f$ en Hertz y una velocidad de propagación $c$ en metros por segundo, el número de onda $k$ (también conocido como número de propagación) se define como:

$$ k = \frac{2 \pi f}{c} $$

​
 
donde:

- $k$ se mide en radianes por metro (rad/m).

- $f$ es la frecuencia en Hertz (Hz).

- $c$ es la velocidad del sonido (u otra onda) en el medio, en metros por segundo (m/s).

In [3]:
def wavenumberToFrequency(k, c=344.0):
    """
    Convert wavenumber (k) to frequency (f).

    Parameters:
        k (float): Wavenumber in radians per meter.
        c (float, optional): Speed of sound in meters per second. Default is 344.0 m/s.

    Returns:
        float: Frequency in Hertz (Hz).
    """
    return 0.5 * k * c / np.pi

### 🔊 Presión sonora compleja a partir del potencial acústico

La **presión sonora** $p(t)$ en un punto, como función del tiempo y derivada del **potencial acústico** $\phi$, se calcula como:

$$
p(t) = i \, \rho \, \omega \, e^{-i \omega t} \, \phi
$$

donde:

- $\rho$: densidad del aire en kilogramos por metro cúbico (kg/m³),
- $\omega = kc$: frecuencia angular en radianes por segundo (rad/s),
- $k$: número de onda en radianes por metro (rad/m),
- $c$: velocidad del sonido en metros por segundo (m/s),
- $\phi$: potencial acústico complejo,
- $t$: tiempo en segundos (s),
- $p(t)$: presión sonora compleja en pascales (Pa).

In [4]:
def soundPressure(k, phi, t=0.0, c=344.0, density=1.205):
    """
    Calculate sound pressure as a complex value.

    Parameters:
        k (float): Wavenumber in radians per meter.
        phi (complex): Acoustic potential.
        t (float, optional): Time in seconds. Default is 0.0.
        c (float, optional): Speed of sound in meters per second. Default is 344.0 m/s.
        density (float, optional): Air density in kilograms per cubic meter. Default is 1.205 kg/m³.

    Returns:
        np.complex64: Sound pressure as a complex value.
    """
    angularVelocity = k * c
    return (1j * density * angularVelocity * np.exp(-1.0j * angularVelocity * t) * phi).astype(np.complex64)

#### ⚡ Intensidad acústica

La **intensidad acústica** promedio $I$ en un medio es:

$$
I = \frac{1}{2} \, \text{Re} \left\{ p^* \, v \right\}
$$

donde:

- $p$: presión sonora compleja (Pa),
- $p^*$: conjugado complejo de la presión sonora,
- $v$: velocidad de partícula compleja (m/s),
- $\text{Re}\{\cdot\}$: parte real del número complejo,
- $I$: intensidad acústica en vatios por metro cuadrado (W/m²).

In [5]:
def AcousticIntensity(pressure, velocity):
    """
    Calculate the acoustic intensity.

    Parameters:
        pressure (complex): Sound pressure.
        velocity (complex): Particle velocity.

    Returns:
        float: Acoustic intensity in watts per square meter (W/m²).
    """
    return 0.5 * (np.conj(pressure) * velocity).real

#### 💡 Fase de la señal

La **fase** $\theta$ de una señal de presión sonora $p$ se calcula como:

$$
\theta = \tan^{-1}\left( \frac{\operatorname{Im}(p)}{\operatorname{Re}(p)} \right)
$$

donde:

- $\operatorname{Re}(p)$: parte real de $p$,
- $\operatorname{Im}(p)$: parte imaginaria de $p$,
- $\theta$: fase en radianes.

In [6]:
def SignalPhase(pressure):
    """
    Calculate the phase of the signal in radians.

    Parameters:
        pressure (complex): Sound pressure.

    Returns:
        float: Phase of the signal in radians.
    """
    return np.arctan2(pressure.imag, pressure.real)

### 🔲 Geometría del contorno cuadrado

La función `Square()` genera un conjunto de **32 vértices** dispuestos sobre el contorno de un **cuadrado de lado 0.1 m**, centrado en el origen (o más específicamente, con esquinas en (0, 0) y (0.1, 0.1)).

#### 📐 Definición de los vértices

Los puntos están distribuidos uniformemente con un espaciado de **0.0125 m** a lo largo de cada lado del cuadrado, siguiendo el orden:

1. Lado izquierdo: de $(0, 0)$ a $(0, 0.1)$
2. Lado superior: de $(0, 0.1)$ a $(0.1, 0.1)$
3. Lado derecho: de $(0.1, 0.1)$ a $(0.1, 0)$
4. Lado inferior: de $(0.1, 0)$ a $(0, 0)$

In [7]:
def Square():
    aVertex = np.array([[0.00, 0.0000], [0.00, 0.0125], [0.00, 0.0250], [0.00, 0.0375],
                         [0.00, 0.0500], [0.00, 0.0625], [0.00, 0.0750], [0.00, 0.0875],
                         
                         [0.0000, 0.10], [0.0125, 0.10], [0.0250, 0.10], [0.0375, 0.10],
                         [0.0500, 0.10], [0.0625, 0.10], [0.0750, 0.10], [0.0875, 0.10],
                         
                         [0.10, 0.1000], [0.10, 0.0875], [0.10, 0.0750], [0.10, 0.0625],
                         [0.10, 0.0500], [0.10, 0.0375], [0.10, 0.0250], [0.10, 0.0125],
                         
                         [0.1000, 0.00], [0.0875, 0.00], [0.0750, 0.00], [0.0625, 0.00],
                         [0.0500, 0.00], [0.0375, 0.00], [0.0250, 0.00], [0.0125, 0.00]], dtype=np.float32)

    aEdge = np.array([[ 0,  1], [ 1,  2], [ 2,  3], [ 3,  4],
                      [ 4,  5], [ 5,  6], [ 6,  7], [ 7,  8],
                      
                      [ 8,  9], [ 9, 10], [10, 11], [11, 12],
                      [12, 13], [13, 14], [14, 15], [15, 16],
                      
                      [16, 17], [17, 18], [18, 19], [19, 20],
                      [20, 21], [21, 22], [22, 23], [23, 24],
                      
                      [24, 25], [25, 26], [26, 27], [27, 28],
                      [28, 29], [29, 30], [30, 31], [31,  0]], dtype=np.int32)

    return aVertex, aEdge

### 📏 Cálculo del vector normal en 2D

La función `Normal2D(pointA, pointB)` calcula un **vector normal unitario** al segmento de línea que une los puntos `pointA` y `pointB` en el plano 2D.

#### 📐 Descripción matemática

Dado un segmento dirigido de $B$ a $A$, el vector tangente unitario $\mathbf{t}$ se define como:

$$
\mathbf{t} = \frac{\mathbf{A} - \mathbf{B}}{\|\mathbf{A} - \mathbf{B}\|}
$$

El vector **normal unitario** $\mathbf{n}$ (obtenido mediante una rotación de 90° en sentido antihorario) es:

$$
\mathbf{n} = 
\begin{bmatrix}
t_y \\
- t_x
\end{bmatrix}
= 
\begin{bmatrix}
\dfrac{A_y - B_y}{\|\mathbf{A} - \mathbf{B}\|} \\
-\dfrac{A_x - B_x}{\|\mathbf{A} - \mathbf{B}\|}
\end{bmatrix}
$$

In [8]:
def Normal2D(pointA, pointB):                  
    diff = pointA - pointB                          
    len = norm(diff)                                
    return np.array([diff[1]/len, -diff[0]/len])   

### 🧮 Cuadratura compleja para integración numérica

La función `ComplexQuad(func, start, end)` realiza la integración numérica de una función `func` en el intervalo $[start, end]$, utilizando una cuadratura ponderada basada en puntos y pesos predefinidos.

#### 📐 Descripción matemática

Para aproximar la integral de la función $f(x)$ en el intervalo $[a, b]$, la fórmula de cuadratura que utiliza los puntos de **Gauss-Legendre** con pesos es:

$$
I \approx \sum_{i=1}^{N} w_i \, f(x_i)
$$

donde:

- $w_i$ son los pesos predefinidos,
- $x_i$ son los puntos de evaluación, distribuidos en el intervalo,
- $N$ es el número de puntos de cuadratura.

En este caso, la cuadratura es una aproximación sobre el segmento $[start, end]$.

In [9]:
def ComplexQuad(func, start, end):
    """
    Calcula la integral numérica de una función utilizando cuadratura ponderada.

    Parámetros:
        func (callable): Función que se desea integrar.
        start (float): Límite inferior de la integración.
        end (float): Límite superior de la integración.

    Retorna:
        float: Aproximación de la integral de func en el intervalo [start, end].
    """
    # Puntos y pesos predefinidos para la cuadratura
    samples = np.array([[0.980144928249, 5.061426814519E-02],                           
                        [0.898333238707, 0.111190517227],                               
                        [0.762766204958, 0.156853322939],                               
                        [0.591717321248, 0.181341891689],                               
                        [0.408282678752, 0.181341891689],                               
                        [0.237233795042, 0.156853322939],                               
                        [0.101666761293, 0.111190517227],                               
                        [1.985507175123E-02, 5.061426814519E-02]], dtype=np.float32)  
    
    # Calculamos el vector de integración
    vec = end - start
    sum = 0.0
    
    # Realizamos la integración ponderada
    for n in range(samples.shape[0]):
        x = start + samples[n, 0] * vec
        sum += samples[n, 1] * func(x)
    
    # Retornamos la suma ponderada multiplicada por la norma del vector
    return sum * norm(vec)


### 📐 Función ComputeL 

La función `ComputeL(k, p, qa, qb, pOnElement)` calcula una **integral de interacción** basada en el parámetro de onda $k$ y los puntos de evaluación $q_a$, $q_b$, y $p$. Dependiendo del valor de $k$, la función realiza diferentes tipos de cálculos.

#### 📊 Descripción de la lógica

1. **Cuando $k = 0$**: La función realiza un cálculo basado en las distancias entre los puntos. La integral se reduce a una expresión logarítmica que involucra las distancias entre los puntos $a$ y $b$ respecto al punto de evaluación $p$.

   $$ L_0 = \frac{1}{2 \pi} \left( \|\mathbf{q_b} - \mathbf{q_a}\| - \left( \|\mathbf{p} - \mathbf{q_a}\| \log(\|\mathbf{p} - \mathbf{q_a}\|) + \|\mathbf{p} - \mathbf{q_b}\| \log(\|\mathbf{p} - \mathbf{q_b}\|) \right) \right) $$

2. **Cuando $k \neq 0$**: Se evalúa una integral que incluye una combinación de la función de Bessel $H_0^{(1)}(k R)$ (de la teoría de propagación de ondas) y un término logarítmico, utilizando una **cuadratura compleja** para calcular la integral sobre el segmento de línea entre $q_a$ y $q_b$.

   $$ L_k = \int_{q_a}^{q_b} \left( \frac{1}{2\pi} \log(\|\mathbf{p} - \mathbf{x}\|) + \frac{1}{4} j H_0^{(1)}(k \|\mathbf{p} - \mathbf{x}\|) \right) \, \text{d}x $$


In [10]:
def ComputeL(k, p, qa, qb, pOnElement):                                                                       
    qab = qb - qa                                                                                                  
    if pOnElement:                                                                                                 
        if k == 0.0:                                                                                               
            ra = norm(p - qa)                                                                                      
            rb = norm(p - qb)                                                                                      
            re = norm(qab)                                                                                         
            return 0.5 / np.pi * (re - (ra * np.log(ra) + rb * np.log(rb)))                                        
        else:                                                                                                      
            def func(x):                                                                                           
                R = norm(p - x)                                                                                    
                return 0.5 / np.pi * np.log(R) + 0.25j * hankel1(0, k * R)                                         
            return ComplexQuad(func, qa, p) + ComplexQuad(func, p, qa) \
                 + ComputeL(0.0, p, qa, qb, True)                                                              
    else:                                                                                                          
        if k == 0.0:                                                                                               
            return -0.5 / np.pi * ComplexQuad(lambda q: np.log(norm(p - q)), qa, qb)                           
        else:                                                                                                      
            return 0.25j * ComplexQuad(lambda q: hankel1(0, k * norm(p - q)), qa, qb)                          
    return 0.0   

### Función ComputeM

La función `ComputeM(k, p, qa, qb, pOnElement)` calcula un término de interacción en función del número de onda $k$ y de las posiciones $p$, $q_a$ y $q_b$.

#### 1. **Cuando $pOnElement = \text{True}$:**
   En este caso, la función devuelve $0$, lo que indica que no se realiza ningún cálculo sobre el elemento.

#### 2. **Cuando $pOnElement = \text{False}$:**
   La función realiza una integración numérica que depende de la distancia entre los puntos y del número de onda $k$. La integral se evalúa de la siguiente manera:

   - **Caso $k = 0$**: La integral es de la forma:

     $$ M_0 = -\frac{1}{2 \pi} \int_{q_a}^{q_b} \frac{\vec{r} \cdot \vec{v}}{\|\vec{r}\|^2} \, \text{d}x $$

     Donde $\vec{r} = \mathbf{p} - \mathbf{x}$ y $\vec{v}$ es el vector normal entre los puntos $q_a$ y $q_b$.

   - **Caso $k \neq 0$**: La integral involucra la **función de Bessel de primer orden** $H_1^{(1)}(k R)$, que describe la propagación de ondas, y se calcula de la siguiente manera:

     $$ M_k = \frac{k}{4} \int_{q_a}^{q_b} \frac{H_1^{(1)}(k R) \, \vec{r} \cdot \vec{v}}{R} \, \text{d}x $$

     Aquí, $R = \|\mathbf{p} - \mathbf{x}\|$ es la distancia entre los puntos $p$ y $x$.

In [11]:
def ComputeM(k, p, qa, qb, pOnElement):                                                                       
    qab = qb - qa                                                                                                  
    vecq = Normal2D(qa, qb)                                                                                    
    if pOnElement:                                                                                                 
        return 0.0                                                                                                 
    else:                                                                                                          
        if k == 0.0:                                                                                               
            def func(x):                                                                                           
                r = p - x                                                                                          
                return np.dot(r, vecq) / np.dot(r, r)                                                              
            return -0.5 / np.pi * ComplexQuad(func, qa, qb)                                                    
        else:                                                                                                      
            def func(x):                                                                                           
                r = p - x                                                                                          
                R = norm(r)                                                                                        
                return hankel1(1, k * R) * np.dot(r, vecq) / R                                                     
            return 0.25j * k * ComplexQuad(func, qa, qb)                                                       
    return 0.0              

In [12]:
def ComputeMt(k, p, vecp, qa, qb, pOnElement):                                                                
    qab = qb - qa                                                                                                  
    if pOnElement:                                                                                                 
        return 0.0                                                                                                 
    else:                                                                                                          
        if k == 0.0:                                                                                               
            def func(x):                                                                                           
                r = p - x                                                                                          
                return np.dot(r, vecp) / np.dot(r, r)                                                              
            return -0.5 / np.pi * ComplexQuad(func, qa, qb)                                                    
        else:                                                                                                      
            def func(x):                                                                                           
                r = p - x                                                                                          
                R = norm(r)                                                                                        
                return hankel1(1, k * R) * np.dot(r, vecp) / R                                                     
            return -0.25j * k * ComplexQuad(func, qa, qb)                                                      
                                                                                                                   
def ComputeN(k, p, vecp, qa, qb, pOnElement):                                                                 
    qab = qb- qa                                                                                                   
    if pOnElement:                                                                                                 
        ra = norm(p - qa)                                                                                          
        rb = norm(p - qb)                                                                                          
        re = norm(qab)                                                                                             
        if k == 0.0:                                                                                               
            return -(1.0 / ra + 1.0 / rb) / (re * 2.0 * np.pi) * re                                                
        else:                                                                                                      
            vecq = Normal2D(qa, qb)                                                                            
            k2 = k * k                                                                                             
            def func(x):                                                                                           
                r = p - x                                                                                          
                R2 = np.dot(r, r)                                                                                  
                R = np.sqrt(R2)                                                                                    
                drdudrdn = -np.dot(r, vecq) * np.dot(r, vecp) / R2                                                 
                dpnu = np.dot(vecp, vecq)                                                                          
                c1 =  0.25j * k / R * hankel1(1, k * R)                                  - 0.5 / (np.pi * R2)      
                c2 =  0.50j * k / R * hankel1(1, k * R) - 0.25j * k2 * hankel1(0, k * R) - 1.0 / (np.pi * R2)      
                c3 = -0.25  * k2 * np.log(R) / np.pi                                                               
                return c1 * dpnu + c2 * drdudrdn + c3                                                              
            return ComputeN(0.0, p, vecp, qa, qb, True) - 0.5 * k2 * ComputeL(0.0, p, qa, qb, True) \
                 + ComplexQuad(func, qa, p) + ComplexQuad(func, p, qb)                                     
    else:                                                                                                          
        sum = 0.0j                                                                                                 
        vecq = Normal2D(qa, qb)                                                                                
        un = np.dot(vecp, vecq)                                                                                    
        if k == 0.0:                                                                                               
            def func(x):                                                                                           
                r = p - x                                                                                          
                R2 = np.dot(r, r)                                                                                  
                drdudrdn = -np.dot(r, vecq) * np.dot(r, vecp) / R2                                                 
                return (un + 2.0 * drdudrdn) / R2                                                                  
            return 0.5 / np.pi * ComplexQuad(func, qa, qb)                                                     
        else:                                                                                                      
            def func(x):                                                                                           
                r = p - x                                                                                          
                drdudrdn = -np.dot(r, vecq) * np.dot(r, vecp) / np.dot(r, r)                                       
                R = norm(r)                                                                                        
                return hankel1(1, k * R) / R * (un + 2.0 * drdudrdn) - k * hankel1(0, k * R) * drdudrdn            
            return 0.25j * k * ComplexQuad(func, qa, qb)        

In [13]:
def SolveLinearEquation(cls, Ai, Bi, ci, alpha, beta, f):
    A = np.copy(Ai)
    B = np.copy(Bi)
    c = np.copy(ci)

    x = np.empty(c.size, dtype=np.complex128)
    y = np.empty(c.size, dtype=np.complex128)

    gamma = np.linalg.norm(B, np.inf) / np.linalg.norm(A, np.inf)
    swapXY = np.empty(c.size, dtype=bool)
    for i in range(c.size):
        if np.abs(beta[i]) < gamma * np.abs(alpha[i]):
            swapXY[i] = False
        else:
            swapXY[i] = True

    for i in range(c.size):
        if swapXY[i]:
            for j in range(alpha.size):
                c[j] += f[i] * B[j,i] / beta[i]
                B[j, i] = -alpha[i] * B[j, i] / beta[i]
        else:
            for j in range(alpha.size):
                c[j] -= f[i] * A[j, i] / alpha[i]
                A[j, i] = -beta[i] * A[j, i] / alpha[i]

    A -= B
    y = np.linalg.solve(A, c)

    for i in range(c.size):
        if swapXY[i]:
            x[i] = (f[i] - alpha[i] * y[i]) / beta[i]
        else:
            x[i] = (f[i] - beta[i] * y[i]) / alpha[i]

    for i in range(c.size):
        if swapXY[i]:
            temp = x[i]
            x[i] = y[i]
            y[i] = temp

    return x, y

In [14]:
aVertex,aElement = Square()

In [15]:
def computeBoundaryMatricesInterior(k, mu):

    aCenters = 0.5 * (aVertex[aElement[:, 0]] + aVertex[aElement[:, 1]])
    # lenght of the boundary elements (for the 3d shapes this is replaced by aArea
    aLength = np.linalg.norm(aVertex[aElement[:, 0]] - aVertex[aElement[:, 1]])
    A = np.empty((aElement.shape[0], aElement.shape[0]), dtype=complex)
    B = np.empty(A.shape, dtype=complex)

    for i in range(aElement.shape[0]):
        pa = aVertex[aElement[i, 0]]
        pb = aVertex[aElement[i, 1]]
        pab = pb - pa
        center = 0.5 * (pa + pb)
        centerNormal = Normal2D(pa, pb)
        for j in range(aElement.shape[0]):
            qa = aVertex[aElement[j, 0]]
            qb = aVertex[aElement[j, 1]]

            elementL  = ComputeL(k, center, qa, qb, i==j)
            elementM  = ComputeM(k, center, qa, qb, i==j)
            elementMt = ComputeMt(k, center, centerNormal, qa, qb, i==j)
            elementN  = ComputeN(k, center, centerNormal, qa, qb, i==j)
            
            A[i, j] = elementL + mu * elementMt
            B[i, j] = elementM + mu * elementN


            # interior variant, signs are reversed for exterior
            A[i,i] -= 0.5 * mu
            B[i,i] += 0.5

In [16]:
def BoundarySolution(parent, k, aPhi, aV):
    """
    Devuelve una cadena formateada con los datos de la solución en la frontera.

    Parámetros:
        parent: objeto con atributos 'density' (kg/m^3) y 'c' (velocidad del sonido en m/s).
        k (float): número de onda.
        aPhi (np.ndarray): potencial complejo en cada punto de la frontera.
        aV (np.ndarray): velocidad compleja en cada punto de la frontera.

    Retorna:
        str: reporte formateado con densidad, velocidad del sonido, presión, intensidad, etc.
    """
    res =  "Density of medium:      {} kg/m^3\n".format(parent.density)
    res += "Speed of sound:         {} m/s\n".format(parent.c)
    res += "Wavenumber (Frequency): {} ({} Hz)\n\n".format(k, wavenumberToFrequency(k))
    res += "index          Potential                   Pressure                    Velocity              Intensity\n"
    for i in range(aPhi.size):
        pressure = soundPressure(k, aPhi[i], c=parent.c, density=parent.density)
        intensity = AcousticIntensity(pressure, aV[i])
        res += "{:5d}  {: 1.4e}+ {: 1.4e}i   {: 1.4e}+ {: 1.4e}i   {: 1.4e}+ {: 1.4e}i    {: 1.4e}\n".format(
            i+1, aPhi[i].real, aPhi[i].imag,
            pressure.real, pressure.imag,
            aV[i].real, aV[i].imag,
            intensity
        )
    return res

In [17]:
def solveInteriorBoundary(self, k, boundaryCondition, boundaryIncidence, mu = None):
    mu = mu or (1j / (k + 1))
    assert boundaryCondition.f.size == self.aElement.shape[0]
    A, B = computeBoundaryMatricesInterior(k, mu)
    c = np.empty(aElement.shape[0], dtype=np.complex128)
    for i in range(aElement.shape[0]):
        # Note, the only difference between the interior solver and this
        # one is the sign of the assignment below.
        c[i] = boundaryIncidence.phi[i] + mu * boundaryIncidence.v[i]

    phi, v = SolveLinearEquation(B, A, c,
                                        boundaryCondition.alpha,
                                        boundaryCondition.beta,
                                        boundaryCondition.f)
    return BoundarySolution(k, phi, v)

In [18]:
class Medium:
    def __init__(self, c=344.0, density=1.205):
        self.c = c
        self.density = density

parent = Medium()

In [24]:
size = aElement.shape[0]
alpha = np.empty(size, dtype = np.complex64)
beta  = np.empty(size, dtype = np.complex64)
f     = np.empty(size, dtype = np.complex64)
phi = np.empty(size, dtype = np.complex64)
v   = np.empty(size, dtype = np.complex64)